<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#Naive-Bayes" data-toc-modified-id="Naive-Bayes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Naive Bayes</a></span></li><li><span><a href="#Decision-Tree" data-toc-modified-id="Decision-Tree-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Decision Tree</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from zeugma.embeddings import EmbeddingTransformer
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
nltk.download('stopwords')
nltk.download('punkt')
def preprocess(data):
    '''                                                                         
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution   
    '''
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°\
£€\×™√²—–&'
    def clean_special_chars(text, punct):
        for p in punct:
            text = text.replace(p, ' ')
        return text

    data = clean_special_chars(str(data), punct)
    data = data.split()
    stop_words = set(stopwords.words('english'))
    cleaned = [word for word in data if word not in stop_words]
    return " ".join(cleaned)

[nltk_data] Downloading package stopwords to /home/danni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/danni/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
cleaned = True
if cleaned:
    train_file = "preprocessed_train.csv"
    df = pd.read_csv(train_file)
else:
    train_file = "train.csv"
    df = pd.read_csv(train_file)
    df['clean_text'] = df['comment_text'].apply(lambda x: preprocess(x))
    df.to_csv("preprocessed_train.csv")

In [4]:
# evluation metrics
# From baseline kernel
from sklearn import metrics
from sklearn.metrics import accuracy_score
def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]>0.5
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)



SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]>0.5]
    return compute_auc((subgroup_examples[label]>0.5), subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]>0.5) & (df[label]<=0.5)]
    non_subgroup_positive_examples = df[(df[subgroup]<=0.5) & (df[label]>0.5)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]>0.5) & (df[label]>0.5)]
    non_subgroup_negative_examples = df[(df[subgroup]<=0.5) & (df[label]<=0.5)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]>0.5])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

In [5]:
#Split into training and test data
train, test = train_test_split(df, test_size=0.2, random_state = 42)
x_train = train['clean_text'].astype(str)
x_test = test['clean_text'].astype(str)
y_train = np.where(train['target'] >= 0.5, 1, 0)
y_test = np.where(test['target'] >= 0.5, 1, 0)
#Encode training data with glove vectors
glove = EmbeddingTransformer('glove')
x_train = glove.transform(x_train)

### Logistic Regression

In [13]:
#Fit LR model
MODEL_NAME = 'LR'
target_model = LogisticRegression(C=5, random_state=42, solver='sag', max_iter=1000, n_jobs=-1)
target_model.fit(x_train, y_train)
x_test_glove = glove.transform(x_test)
predictions = target_model.predict_proba(x_test_glove)[:,1]

val = pd.DataFrame.from_dict({
    'id': test['id'],
    'prediction': predictions
})

In [14]:
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
MODEL_NAME = 'LR'
test[MODEL_NAME]= val["prediction"]
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test, identity_columns, MODEL_NAME, 'target')
binary_labels = np.where(test[TOXICITY_COLUMN] >= 0.5, 1, 0)
binary_preds = np.where(test[MODEL_NAME] >= 0.5, 1, 0)
print("accuracy: ", (accuracy_score(binary_labels, binary_preds)))
print("final auc ", get_final_metric(bias_metrics_df, calculate_overall_auc(test, MODEL_NAME)))
bias_metrics_df

accuracy:  0.9174568875960939
final auc  0.7587788675469529


,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
6,0.825947,0.668087,black,0.723679,2729
5,0.758089,0.760505,muslim,0.724064,3940
7,0.843873,0.646589,white,0.728359,4661
2,0.842161,0.653064,homosexual_gay_or_lesbian,0.734267,2104
8,0.789674,0.739740,psychiatric_or_mental_illness,0.749666,861
1,0.839600,0.690624,female,0.754354,10136
4,0.762437,0.778595,jewish,0.754917,1431
0,0.852224,0.676963,male,0.763339,8025
3,0.733759,0.833909,christian,0.791629,7011


### Naive Bayes

In [28]:
from sklearn.naive_bayes import GaussianNB

#Fit NB model
MODEL_NAME = 'NB'
target_model = GaussianNB()
target_model.fit(x_train, y_train)
x_test_glove = glove.transform(x_test)
predictions = target_model.predict_proba(x_test_glove)[:,1]

val = pd.DataFrame.from_dict({
    'id': test['id'],
    'prediction': predictions
})

In [36]:
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
MODEL_NAME = 'NB'
test[MODEL_NAME]= val["prediction"]
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test, identity_columns, MODEL_NAME, 'target')
binary_labels = np.where(test[TOXICITY_COLUMN] >= 0.5, 1, 0)
binary_preds = np.where(test[MODEL_NAME] >= 0.5, 1, 0)
print("accuracy: ", (accuracy_score(binary_labels, binary_preds)))
print("final auc ", get_final_metric(bias_metrics_df, calculate_overall_auc(test, MODEL_NAME)))
bias_metrics_df

accuracy:  0.8987658425098691
final auc  0.714034769963206


,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
8,0.656839,0.721844,psychiatric_or_mental_illness,0.623098,861
2,0.796367,0.603896,homosexual_gay_or_lesbian,0.657850,2104
6,0.793092,0.621605,black,0.672298,2729
5,0.723784,0.714569,muslim,0.677670,3940
7,0.814156,0.601035,white,0.682683,4661
3,0.741244,0.732741,christian,0.721359,7011
4,0.765865,0.709661,jewish,0.728922,1431
1,0.836923,0.630849,female,0.733822,10136
0,0.849821,0.607552,male,0.736357,8025


### Decision Tree

In [6]:
#### using linear kernel 
from sklearn import tree

#Fit svm model
MODEL_NAME = 'tree'
target_model = tree.DecisionTreeClassifier()

target_model.fit(x_train, y_train)
x_test_glove = glove.transform(x_test)
predictions = target_model.predict_proba(x_test_glove)[:,1]

val = pd.DataFrame.from_dict({
    'id': test['id'],
    'prediction': predictions
})

In [7]:
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
MODEL_NAME = 'svm'
test[MODEL_NAME]= val["prediction"]
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test, identity_columns, MODEL_NAME, 'target')
binary_labels = np.where(test[TOXICITY_COLUMN] >= 0.5, 1, 0)
binary_preds = np.where(test[MODEL_NAME] >= 0.5, 1, 0)
print("accuracy: ", (accuracy_score(binary_labels, binary_preds)))
print("final auc ", get_final_metric(bias_metrics_df, calculate_overall_auc(test, MODEL_NAME)))
bias_metrics_df

accuracy:  0.8630071334579957
final auc  0.5748473499533646


,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
5,0.554963,0.581225,muslim,0.553456,3940
2,0.587537,0.554199,homosexual_gay_or_lesbian,0.561198,2104
7,0.595721,0.545819,white,0.561731,4661
6,0.592515,0.558921,black,0.571647,2729
3,0.561114,0.594033,christian,0.574780,7011
4,0.567808,0.588183,jewish,0.575537,1431
1,0.586721,0.570366,female,0.576261,10136
0,0.594696,0.563981,male,0.578675,8025
8,0.582084,0.592325,psychiatric_or_mental_illness,0.594115,861
